In [28]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI()

In [29]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("./document.txt")

docs = loader.load_and_split(text_splitter=splitter)

docs

[Document(page_content='Chapter 3\n\'There are three stages in your reintegration,\' said O\'Brien. \'There is\nlearning, there is understanding, and there is acceptance. It is time for\nyou to enter upon the second stage.\'\nAs always, Winston was lying flat on his back. But of late his bonds were\nlooser. They still held him to the bed, but he could move his knees a\nlittle and could turn his head from side to side and raise his arms from\nthe elbow. The dial, also, had grown to be less of a terror. He could\nevade its pangs if he was quick-witted enough: it was chiefly when he\nshowed stupidity that O\'Brien pulled the lever. Sometimes they got through\na whole session without use of the dial. He could not remember how many\nsessions there had been. The whole process seemed to stretch out over a\nlong, indefinite time--weeks, possibly--and the intervals between the\nsessions might sometimes have been days, sometimes only an hour or two.\n\'As you lie there,\' said O\'Brien, \'you ha

In [30]:
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS

cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

In [31]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

prompt = ChatPromptTemplate.from_messages(
    [
      (
        "system",
        "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
      ),
      MessagesPlaceholder(variable_name="history"),
      ("human", "{question}"),
    ]
)

def load_memory(_):
  return memory.load_memory_variables({})["history"]

chain = (
  {
    "context": retriever,
    "question": RunnablePassthrough()
  }
  | RunnablePassthrough.assign(history=load_memory)
  | prompt
  | llm
)

def invoke_chain(question):
  result = chain.invoke(question)
  memory.save_context(
      {"input": question},
      {"output": result.content}
  )
  print(result)

In [32]:
invoke_chain("Is Aaronson guilty?")

content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [33]:
invoke_chain("What message did he write in the table?")

content='He wrote in large clumsy capitals: FREEDOM IS SLAVERY.'


In [34]:
invoke_chain("Who is Julia?")

content='Julia is a character who Winston was romantically involved with and who shared his rebellious views against the Party.'


In [35]:
invoke_chain("Is Aaronson guilty?")

content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [36]:
invoke_chain("What message did he write in the table?")

content='He wrote in large clumsy capitals: FREEDOM IS SLAVERY.'


In [37]:
invoke_chain("Who is Julia?")

content='Julia is a character who Winston was romantically involved with and who shared his rebellious views against the Party.'


In [38]:
invoke_chain("3번째 질문과 답변이 뭐야?")

content='제가 알고 있는 정보는 "Julia는 파티에 반대하는 반항적인 견해를 공유한 윈스턴과 로맨틱하게 연관된 캐릭터입니다." 입니다.'
